# NAIVE BAYES
We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [39]:
import os 
import io
import numpy as np
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('/home/william/Dev/ml/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('/home/william/Dev/ml/emails/ham', 'ham'))


In [40]:
data.head(10)

,message,class
/home/william/Dev/ml/emails/spam/00347.0958e79c14164f0f902d863f41156c0b,Opportunity is knocking. Why?\n\n\n\nBecause m...,spam
/home/william/Dev/ml/emails/spam/00276.a6e447390e371ddba7cee092bb0ec98f,"<html>\n\n\n\n<head>\n\n<meta http-equiv=3D""Co...",spam
/home/william/Dev/ml/emails/spam/00357.b523d4209d633d6fdf86b93bc19e3aa2,<html>\n\n<head>\n\n<title>Digital Publishing ...,spam
/home/william/Dev/ml/emails/spam/00270.5dcd9ce3be2992222b9038d7bf75a23a,"Dear Partner to be,\n\n\n\nFirst, I must apolo...",spam
/home/william/Dev/ml/emails/spam/00381.7d436777379ad18167e4614190b206cf,================This Is Definitely the Answer ...,spam
/home/william/Dev/ml/emails/spam/00070.ab34b6c044a55bef3d6c1f64b7521773,When America's top companies compete for your ...,spam
/home/william/Dev/ml/emails/spam/00425.1434e0ab4e5235b64825b4c2a0999d76,<HTML><HEAD><TITLE>Hi i'm Rita !!!</TITLE>\n\n...,spam
/home/william/Dev/ml/emails/spam/00063.2334fb4e465fc61e8406c75918ff72ed,IS YOUR BUSINESS MAKING MONEY!\n\nSet Up To Ac...,spam
/home/william/Dev/ml/emails/spam/00182.1b9ba0f95506a6f2bf256f40fad0687d,All our mailings are sent complying to the pro...,spam
/home/william/Dev/ml/emails/spam/00316.311d11f764c6e452b2f0208b53b94ea2,NEW - NEW - NEW IN THE MARKET\n\nJUST RELEAS...,spam


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go! It's just that easy.

In [41]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Let's give it a try

In [42]:
examples = ['Free opportunity knocking!!', "Hi Will, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')

In [43]:
examples = ['Free viagra!!!', "Hello Will, care for a game of chess this evening?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')

Our data set is small, so our spam classifier isn't actually very good.